In [1]:
from collections import defaultdict, Counter
import pandas as pd
import spacy
from spacy.de import German

In [2]:
parser = German()

In [3]:
text = open('example_book.txt').read()

from: 

http://stackoverflow.com/questions/37253326/how-to-find-the-most-common-words-using-spacy/37253693#37253693

In [4]:
doc = parser(text)

In [5]:
pos_counts = defaultdict(Counter)

In [6]:
## token.pos -- part of speech

for token in doc:
    pos_counts[token.pos][token.lower] += 1

In [7]:


pos_, count_, strings_ = [], [], []
for pos_id, counts in sorted(pos_counts.items()):
    pos = doc.vocab.strings[pos_id]
    for orth_id, count in counts.most_common():
        pos_.append(pos)
        count_.append(count)
        strings_.append(doc.vocab.strings[orth_id])

In [12]:
text_word_counts = pd.DataFrame({'pos':pos_, 'count': count_, 'word': strings_})

In [33]:
text_word_counts.groupby('pos').agg({'count': sum})['count'].sort_values()

pos
INTJ        16
NUM        286
X          307
SCONJ     1786
PROPN     2451
PART      3243
CONJ      3758
AUX       4457
ADJ       5865
ADP       6223
SPACE     8170
ADV       9623
DET      10669
PRON     11138
VERB     12836
NOUN     13596
PUNCT    18616
Name: count, dtype: int64

In [16]:
text_word_counts.sort_values('count', ascending=False).head(10)

,count,pos,word
6332,8065,PUNCT,","
9456,6562,SPACE,\n
6333,3924,PUNCT,.
2414,2678,CONJ,und
2436,2303,DET,die
6334,2179,PUNCT,«
5930,2089,PRON,sie
2437,1935,DET,der
6335,1925,PUNCT,»
9457,1553,SPACE,\n\n


In [43]:
top_500 = (text_word_counts[~text_word_counts.pos.isin(['PUNCT', 'SPACE', 'X'])].
   sort_values('count',ascending=False).
   iloc[0:500]
)

In [47]:
top_500_words = top_500.word.unique()

In [52]:
top_500_words[0:100] ## need to remove entities?? or it doesn't matter?? -- people are in text etc
                     ## will just have main characters and so on included??

array(['und', 'die', 'sie', 'der', 'nicht', 'er', 'es', 'in', 'ich', 'so',
       'war', 'den', 'das', 'sich', 'ist', 'mit', 'dem', 'ein', 'sagte',
       'daß', 'du', 'zu', 'auch', 'aber', 'hatte', 'auf', 'von', 'noch',
       'wenn', 'doch', '’s', 'frau', 'an', 'nur', 'mutter', 'schon', 'im',
       'ihm', 'eine', 'was', 'da', 'wie', 'nun', 'hat', 'dann', 'für',
       'wieder', 'man', 'ihr', 'nach', 'wir', 'ihn', 'einen', 'ganz',
       'jetzt', 'mir', 'ja', 'kam', 'bei', 'des', 'am', 'sah', 'seine',
       'vater', 'gar', 'kind', 'als', 'habe', 'greiner', 'mann', 'wurde',
       'aus', 'haben', 'sein', 'ging', 'mehr', 'vor', 'wohl', 'wollte',
       'fragte', 'kann', 'wird', 'nichts', 'waren', 'konnte', 'denn',
       'dir', 'über', 'hermann', 'immer', 'kinder', 'kleine', 'sind',
       'haus', 'herr', 'einmal', 'ihre', 'will', 'zum', 'wo'], dtype=object)

In [62]:
sentences = doc.sents

need to clean stuff out of sentences - large spaces etc

In [65]:
ex = next(sentences)

In [66]:
ex

Ja, er hatte
sich mit allerlei fleißigen und faulen Schülern redlich geplagt, das
ganze Jahr hindurch, hatte Violin- und Flöten-, Klavier- und
Zitherstunden gegeben von frühmorgens bis spät abends.

In [90]:
ex

Ja, er hatte
sich mit allerlei fleißigen und faulen Schülern redlich geplagt, das
ganze Jahr hindurch, hatte Violin- und Flöten-, Klavier- und
Zitherstunden gegeben von frühmorgens bis spät abends.

In [88]:
example_sentence = ex.text.replace('\n',' ')

In [92]:
ex_doc = parser(example_sentence)

In [127]:
ex_doc

Ja, er hatte sich mit allerlei fleißigen und faulen Schülern redlich geplagt, das ganze Jahr hindurch, hatte Violin- und Flöten-, Klavier- und Zitherstunden gegeben von frühmorgens bis spät abends.

In [117]:
ex_lower_words = [token.lower_ for token in ex_doc]

In [121]:
ex_lower_words

['ja',
 ',',
 'er',
 'hatte',
 'sich',
 'mit',
 'allerlei',
 'fleißigen',
 'und',
 'faulen',
 'schülern',
 'redlich',
 'geplagt',
 ',',
 'das',
 'ganze',
 'jahr',
 'hindurch',
 ',',
 'hatte',
 'violin-',
 'und',
 'flöten-',
 ',',
 'klavier-',
 'und',
 'zitherstunden',
 'gegeben',
 'von',
 'frühmorgens',
 'bis',
 'spät',
 'abends',
 '.']

In [120]:
len([word for word in ex_lower_words if word in top_500_words])

14

In [ ]:
ex_lower_words

In [ ]:
## other chars to replace

# _
# »


In [181]:
punct = [',', '.', ';', ':', '-', '»']

def word_proportion(sentence, word_list):
    sentence_clean = sentence.text.replace('\n', ' ').replace('_', '').replace('»', '').replace('«', '')
    parsed_sentence = parser(sentence_clean)
    ## drop punctuation...
    sent_lower = [token.lower_ for token in parsed_sentence if token not in punct]
    sent_len = len(sent_lower)
    if sent_len <= 2:
        return None
    n_words = len([word for word in sent_lower if word in word_list])
    #print('sentence word list proportion: {:.2f}'.format(n_words / sent_length))
    #print(sentence_clean, end='\n')
    return pd.DataFrame({'sentence': [sentence_clean], 
                         'n_words': [n_words], 
                         'length': [sent_len],
                         'proportion': [n_words / sent_len]})

In [138]:
next(sentences)

So ging Herr
Pfäffling gleich von der Schule aus in die Buchhandlung, erwarb sich
dort eine Karte vom Fichtelgebirge, und weil er sie schon auf dem Weg
nach Hause studierte, so kam er später heim als sonst und fand die ganze
Familie um den gedeckten Tisch versammelt.

In [157]:
x = next(sentences)
word_proportion(x, top_500_words)

,length,n_words,proportion,sentence
0,11,7,0.636364,und doch können auch die Reisen im Geist jäh u...


In [183]:
sentences = doc.sents
scored_sentences = pd.concat(word_proportion(sent, top_500_words) for sent in sentences)
scored_sentences = scored_sentences.sort_values('proportion', ascending=False).reset_index(drop=True)

In [188]:
scored_sentences

,length,n_words,proportion,sentence
0,5,5,1.000000,Jetzt endlich waren sie an
1,3,3,1.000000,Herr und Frau
2,3,3,1.000000,Jetzt mußte sein
3,3,3,1.000000,Bis ans Haus
4,5,5,1.000000,Darf ich’s denn nicht
5,5,5,1.000000,Wenn nur die Patin so
6,3,3,1.000000,Bei dem Wort
7,4,4,1.000000,Man wird ihn doch
8,4,4,1.000000,Der Herr soll zu
9,3,3,1.000000,So etwas muß
